In [5]:
edges = read.csv('../data/2011/edgelist.csv')
nodes = read.csv('../data/2011/nodelist.csv')

In [6]:
nodes = transform(nodes, landlocked=as.factor(landlocked))
n_countries = dim(nodes)[1]

In [7]:
nodes$gdp_us_dollar <- log(nodes$gdp_us_dollar)
nodes$area <- log(nodes$area)
nodes$population <- log(nodes$population)
nodes$gdp_per_capita <- log(nodes$gdp_per_capita)

In [8]:
nodes$gdp_us_dollar = as.numeric(scale(nodes$gdp_us_dollar))
nodes$gdp_growth = as.numeric(scale(nodes$gdp_growth))
nodes$inflation_rate = as.numeric(scale(nodes$inflation_rate))
nodes$population = as.numeric(scale(nodes$population))
nodes$gdp_per_capita = as.numeric(scale(nodes$gdp_per_capita))
nodes$agriculture_forestry_fishing_of_gdp = as.numeric(scale(nodes$agriculture_forestry_fishing_of_gdp))
nodes$industry_of_gdp = as.numeric(scale(nodes$industry_of_gdp))
nodes$merchandise_of_gdp = as.numeric(scale(nodes$merchandise_of_gdp))
nodes$net_barter_of_trade = as.numeric(scale(nodes$net_barter_of_trade))
nodes$foreign_direct_investment_inflows = as.numeric(scale(nodes$foreign_direct_investment_inflows))

In [9]:
nodes = subset(nodes, select = -c(population, area, gdp_per_capita))

In [10]:
head(nodes, 3)

,country_iso3,foreign_direct_investment_inflows,continent,net_barter_of_trade,inflation_rate,langoff_1,gdp_growth,agriculture_forestry_fishing_of_gdp,industry_of_gdp,gdp_us_dollar,merchandise_of_gdp,landlocked
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,AFG,-0.3009823,Asia,0.2850790,0.9180415,Persian,-0.52191825,1.0662655,-0.3946306,-0.3714823,-0.8762355,1
2,AGO,-0.3728506,Africa,2.1918886,1.1996211,Portuguese,-0.04976185,-0.4800047,1.8415661,0.4290518,0.2014880,0
3,ALB,-0.2776986,Europe,-0.6803136,-0.4871133,Albanian,-0.19341918,0.5896411,-0.2774569,-0.5121953,-0.3820768,0


In [11]:
numerical_columns = colnames(nodes)[unlist(lapply(nodes, is.numeric))]
categorical_columns = colnames(nodes)[!unlist(lapply(nodes, is.numeric))]
categorical_columns = categorical_columns[categorical_columns != 'country_iso3']

In [12]:
dyads = matrix(0, nrow = n_countries, ncol = n_countries)
nodecovs = array(
    rep(0, length(numerical_columns)*n_countries*n_countries), 
    c(length(numerical_columns), n_countries, n_countries)
)
absdiffs = array(
    rep(0, length(numerical_columns)*n_countries*n_countries), 
    c(length(numerical_columns), n_countries, n_countries)
)
nodematchs = array(
    rep(0, length(categorical_columns)*n_countries*n_countries), 
    c(length(categorical_columns), n_countries, n_countries)
)

In [13]:
for (i in 1:n_countries) {
    for (j in 1:n_countries) {
        if (sum(edges$source == nodes$country_iso3[i] & edges$target == nodes$country_iso3[j])) {
            dyads[i, j] = 1
        }    
        for (k in 1:length(numerical_columns)) {
            nodecovs[k, i, j] = nodes[i, numerical_columns[k]] + nodes[j, numerical_columns[k]]
            absdiffs[k, i, j] = abs(nodes[i, numerical_columns[k]] - nodes[j, numerical_columns[k]])
        }
        for (k in 1:length(categorical_columns)) {
            if (nodes[i, categorical_columns[k]] == nodes[j, categorical_columns[k]]) {
                 nodematchs[k, i, j] = 1   
            }
        }
    }
}

In [14]:
df = data.frame(
    edge = as.vector(t(dyads))
)

In [11]:
for (k in 1:length(numerical_columns)) {
    df[, paste('nodecov-',numerical_columns[k], sep = '')] = as.vector(t(nodecovs[k,,]))
}
for (k in 1:length(numerical_columns)) {
    df[, paste('absdiff-',numerical_columns[k], sep = '')] = as.vector(t(absdiffs[k,,]))
}
for (k in 1:length(categorical_columns)) {
    df[, paste('nodematch-',categorical_columns[k], sep = '')] = as.vector(t(nodematchs[k,,]))
}

In [12]:
self_loops_indices = matrix(0, nrow = n_countries)
for (i in 1:n_countries) {
    self_loops_indices[i] = 1 + n_countries*(i - 1) + (i - 1)
}

In [13]:
df = df[-self_loops_indices, ]

In [16]:
set.seed(19746)
model = glm(edge ~ ., data = df, family = 'binomial', control=glm.control(maxit=50))

In [17]:
summary(model)


Call:
glm(formula = edge ~ ., family = "binomial", data = df, control = glm.control(maxit = 50))

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.99957    0.01859  -107.6   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 20139  on 27555  degrees of freedom
Residual deviance: 20139  on 27555  degrees of freedom
AIC: 20141

Number of Fisher Scoring iterations: 4


In [18]:
p_values = coef(summary(model))[,'Pr(>|z|)']
p_values = unname(p_values)
names = colnames(df)
names[1] = 'intercept'

In [19]:
significant_indices = which(p_values < .1)

In [20]:
p_values = p_values[significant_indices]
names = names[significant_indices]

In [21]:
result_df = data.frame(
    effect = names, 
    significance = p_values
)

In [22]:
write.csv(result_df, '../reports/gravity_model_results.csv', row.names = F)